In [82]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from mlpp.hawkesnoparam.estim import Estim
import mlpp.pp.hawkes as hk
from whma.metrics import rel_err, rank_corr, mse_K_part
from whma.cumulants import Cumulants, get_C_claw, get_K_th
from numba import autojit

In [83]:
d = 10
mus = 0.0005 * np.ones(d)
Alpha_true = np.zeros((d,d))
Beta = np.zeros((d,d))
#Alpha_true = np.ones((d,d)) / 15
for i in range(5):
    for j in range(5):
        if i <= j:
            Alpha_true[i][j] = 1.
            Beta[i][j] = 10.
for i in range(5,10):
    for j in range(5,10):
        if i >= j:
            Alpha_true[i][j] = 1.
            Beta[i][j] = 1.
Alpha_true /= 6
#Alpha_truth = np.ones((d,d)) / 15

In [84]:
_, s, _ = np.linalg.svd(Alpha_true)
print(s.max())
assert s.max() < 1, "Alpha_true cannot generate a stable Hawkes process"

0.585556181944


In [85]:
kernels = [[hk.HawkesKernelExp(a, b) for (a, b) in zip(a_list, b_list)] for (a_list, b_list) in zip(Alpha_true, Beta)]
h = hk.Hawkes(kernels=kernels, mus=list(mus))
T_max = 10000000
h.simulate(T_max)
estim = Estim(h, n_threads=8, hDelta=hDelta, hMax=hMax)

## Number of points

In [86]:
for i in range(h.dimension):
    print("i="+str(i)+"     "+str(len(h.get_full_process()[i])))

i=0     12416
i=1     10400
i=2     8531
i=3     7169
i=4     6078
i=5     6116
i=6     7343
i=7     8801
i=8     10486
i=9     12422


# the class *Cumulants*

In [87]:
N = h.get_full_process()
cumul = Cumulants(N)

### compute useful quantities to estimate the full tensor K

In [88]:
cumul.compute_all()

### compute useful quantities to estimate the full tensor K

In [89]:
cumul.compute_all_part()

### compute theoretical - i.e. depending on $\Sigma$ and $R_{true}$ - cumulants

In [90]:
from scipy.linalg import inv
R_true = inv(np.eye(d)-Alpha_true)
cumul.set_C_th(R_true)
cumul.set_K_th(R_true)
cumul.set_K_part_th(R_true)

# Estimation of C (we also compare with C_from_claw)

In [91]:
C_from_claw = get_C_claw(estim)
print(rel_err(cumul.C_th,C_from_claw))

0.0174566452799


In [92]:
print(rel_err(cumul.C_th,cumul.C))

0.017157882074


# Estimation of K

In [93]:
hMax_ = hMax
cumul.compute_F(H=hMax_)
cumul.set_K()
print(rel_err(cumul.K_th,cumul.K))

0.0242954407317


### Estimation procedure for a smaller $hMax$

In [94]:
hMax_ = hMax*.4
cumul.compute_F(H=hMax_)
cumul.set_K()
print(rel_err(cumul.K_th,cumul.K))

0.0218019652571


# Estimation of K_part

In [95]:
hMax_ = hMax
cumul.compute_B(H=hMax_)
cumul.set_K_part()
print(rel_err(cumul.K_part_th,cumul.K_part))

0.0400872834456


### Estimation procedure for a smaller $hMax$

In [96]:
hMax_ = hMax*.4
cumul.compute_B(H=hMax_)
cumul.set_K_part()
print(rel_err(cumul.K_part_th,cumul.K_part))

0.0284253167436
